<a href="https://colab.research.google.com/github/shaoni11/Coursera_Capstone/blob/master/jobathon_updated_may.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Credit card lead prediction
!pip install catboost
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier,StackingClassifier, HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,classification_report, roc_auc_score
from sklearn.preprocessing import LabelEncoder, KBinsDiscretizer, RobustScaler
from sklearn.model_selection import KFold,StratifiedKFold, GroupKFold,train_test_split
import gc
import datetime
from tqdm.notebook import tqdm ,tnrange
import warnings
warnings.filterwarnings("ignore")
np.random.seed(0)
pd.set_option('display.max_columns', 500)


     |████████████████████████████████| 67.3MB 54kB/s 


In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/train_s3TEQDk.csv")
df_test = pd.read_csv("/content/drive/MyDrive/test_mSzZ8RL.csv")
df_total = pd.concat([df_train,df_test],ignore_index=True, sort=False)
print("Shape of datasets: ", df_train.shape, df_test.shape, df_total.shape)

Shape of datasets:  (245725, 11) (105312, 10) (351037, 11)


In [ ]:
df_total['Credit_Product'].fillna('unknown',inplace=True)

In [ ]:
objList = df_total.select_dtypes(include = "object").columns


In [ ]:
#Label Encoding for object to numeric conversion
le = LabelEncoder()
for feat in objList:
  df_total[feat] = le.fit_transform(df_total[feat].astype(str))
  
print (df_total.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351037 entries, 0 to 351036
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   351037 non-null  int64  
 1   Gender               351037 non-null  int64  
 2   Age                  351037 non-null  int64  
 3   Region_Code          351037 non-null  int64  
 4   Occupation           351037 non-null  int64  
 5   Channel_Code         351037 non-null  int64  
 6   Vintage              351037 non-null  int64  
 7   Credit_Product       351037 non-null  int64  
 8   Avg_Account_Balance  351037 non-null  int64  
 9   Is_Active            351037 non-null  int64  
 10  Is_Lead              245725 non-null  float64
dtypes: float64(1), int64(10)
memory usage: 29.5 MB
None


In [ ]:
premium_discretizer = KBinsDiscretizer(n_bins=8, encode='ordinal', strategy='quantile')
df_total['Avg_Account_Balance_bin'] =premium_discretizer.fit_transform(df_total['Avg_Account_Balance'].values.reshape(-1,1)).astype(int)

age_discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
df_total['Age_bin'] =age_discretizer.fit_transform(df_total['Age'].values.reshape(-1,1)).astype(int)
df_total['Vintage_bin'] =age_discretizer.fit_transform(df_total['Vintage'].values.reshape(-1,1)).astype(int)


In [ ]:
train_x_full = df_total[df_total['Is_Lead'].notnull()]
test_x_full = df_total[df_total['Is_Lead'].isnull()]
train_y_full = train_x_full['Is_Lead']
test_id=test_x_full['ID']
del test_x_full['Is_Lead']
del train_x_full['Is_Lead']
del test_x_full['ID']
del train_x_full['ID']
print(train_x_full.shape, train_y_full.shape, test_x_full.shape)

test_x_full.nunique()

(245725, 12) (245725,) (105312, 12)


Gender                         2
Age                           62
Region_Code                   35
Occupation                     4
Channel_Code                   4
Vintage                       66
Credit_Product                 3
Avg_Account_Balance        78315
Is_Active                      2
Avg_Account_Balance_bin        8
Age_bin                       10
Vintage_bin                   10
dtype: int64

In [ ]:
NFOLDS = 20
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=18121995)
pred_test = []
pred_x = []
pred_y = []
j=1

In [ ]:
train_x = train_x_full.copy()
train_y = train_y_full.copy()
test_x = test_x_full.copy()
feature_importance_df = pd.DataFrame()


In [ ]:
cat_features = ['Gender','Region_Code',	'Channel_Code',	'Credit_Product'	,'Is_Active'	,	'Avg_Account_Balance','Age' , 'Vintage']


In [ ]:
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('------- Fold -------:', fold+1)
    model = CatBoostClassifier(n_estimators=1000,random_state=1812195,learning_rate=0.03,eval_metric='AUC',
                            cat_features =cat_features)
    model.fit(train_x.loc[train_ids], train_y.loc[train_ids], 
              eval_set=[(train_x.loc[train_ids], train_y.loc[train_ids]), (train_x.loc[test_ids], train_y.loc[test_ids])],
              verbose=500,
              early_stopping_rounds=200)
    pred_fold = model.predict_proba(train_x.loc[test_ids])[:,-1]
    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    pred_fold_test = model.predict_proba(test_x)[:,-1]
    pred_test.append([float(i) for i  in pred_fold_test])
    print('\n')
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_x.loc[train_ids].columns
    fold_importance_df["importance"] = model.feature_importances_
    fold_importance_df["fold"] = j + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    j=j+1

print("Final ROC-AUC Score:", roc_auc_score(pred_y, pred_x))


------- Fold -------: 1
0:	test: 0.8587186	test1: 0.8523706	best: 0.8523706 (0)	total: 728ms	remaining: 12m 7s
500:	test: 0.8821664	test1: 0.8689264	best: 0.8690244 (325)	total: 3m 36s	remaining: 3m 35s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8690243885
bestIteration = 325

Shrink model to first 326 iterations.


------- Fold -------: 2
0:	test: 0.8543639	test1: 0.8483216	best: 0.8483216 (0)	total: 608ms	remaining: 10m 7s
500:	test: 0.8853070	test1: 0.8685849	best: 0.8686208 (492)	total: 3m 35s	remaining: 3m 34s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8686208289
bestIteration = 492

Shrink model to first 493 iterations.


------- Fold -------: 3
0:	test: 0.8576188	test1: 0.8613511	best: 0.8613511 (0)	total: 627ms	remaining: 10m 26s
500:	test: 0.8808697	test1: 0.8775055	best: 0.8775256 (481)	total: 3m 38s	remaining: 3m 37s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8776563926
bestIteration = 553

Shrink 

In [ ]:
print("Final ROC-AUC Score:", roc_auc_score(pred_y, pred_x))


Final ROC-AUC Score: 0.8745050968198607


In [ ]:

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

vote = final_prediction.mean(axis=1)
final_prediction['Is_Lead'] = vote
final_prediction['ID'] = test_id.values
print(final_prediction.shape)

train_pred_all['cb_pred'] = pred_x
final_prediction_cb = final_prediction.copy()

final_prediction.head()

#train_pred_all['cb_pred'] = pred_x
final_prediction_cb = final_prediction

final_prediction.head()